In [ ]:
# imports
import torch
import os
import configparser
import matplotlib.pyplot as plt

In [ ]:
device = 'cuda'

In [ ]:
def load_experiment_models(config_path):
    """
    Load all .pth files from the specified experiment directory using configuration details from a .conf file.

    :param config_path: Path to the configuration .conf file.
    :return: A list of loaded models or data from the .pth files.
    """
    # Create a ConfigParser object
    config = configparser.ConfigParser()

    # Read the configuration from the .conf file
    config.read(config_path)

    if not os.path.exists(config_path):
        print(f"Configuration file {config_path} does not exist.")
    elif os.path.getsize(config_path) == 0:
        print(f"Configuration file {config_path} is empty.")
    else:
        print(f"Configuration file {config_path} is found and has content.")

    # Print out all sections and keys for debugging
    print("Sections found in config:", config.sections())
    for section in config.sections():
        print(f"Keys in section '{section}':", config.options(section))

    # Extract the base directory and experiment name from the config
    try:
        base_directory = config['EXPERIMENT']['base_directory']
        experiment_name = config['EXPERIMENT']['experiment_name']
    except KeyError as e:
        print(f"Missing configuration key: {e}")
        return []

    # Construct the full path to the experiment directory
    experiment_directory = os.path.join(os.path.join(base_directory, experiment_name), 'models')

    # Initialize a list to store the loaded models or data
    loaded_models = []

    # Check if the directory exists
    if not os.path.isdir(experiment_directory):
        print(f"Directory {experiment_directory} does not exist.")
        return loaded_models

    # Iterate over all files in the experiment directory
    for filename in os.listdir(experiment_directory):
        # Check if the file has a .pth extension
        if filename.endswith('.pth'):
            # Construct the full path to the .pth file
            file_path = os.path.join(experiment_directory, filename)

            # Load the .pth file
            try:
                loaded_model = torch.load(file_path).to(device)

                # Append the loaded model or data to the list
                loaded_models.append(loaded_model)
                print(f"Loaded {filename} successfully.")
            except Exception as e:
                print(f"Failed to load {filename}: {e}")

    return loaded_models

In [ ]:
# Example usage:
config_path = 'configs/experiment1.conf'  # Path to the config file

# Load the models for the specified experiment
models = load_experiment_models(config_path)

In [ ]:
# extracting the densities
N = 100
scale = 1.5

x = torch.linspace(-scale, scale, N)
y = torch.linspace(-scale, scale, N)
z = torch.linspace(-scale, scale, N)

x, y, z = torch.meshgrid((x, y, z))

xyz = torch.cat((x.reshape(-1, 1),
                 y.reshape(-1, 1),
                 z.reshape(-1, 1)), dim=1)

In [ ]:
# arithmetic mean of each point (x,y,z) in our 3D grid
densities = []
for model in models:

    with torch.no_grad():
        _, density = model.forward(xyz.to(device), torch.zeros_like(xyz).to(device))
    
    #density = density.cpu().numpy().reshape(N, N, N)
    density = density.view(N, N, N)
    densities.append(density)

densities_tensor = torch.stack(densities)
mean_density = torch.mean(densities_tensor, dim=0) # calculates the arithmetic mean for every point

In [ ]:
densities[0].shape

In [ ]:
# for all densities across the 7 models, substract the model density from the mean
squared_diffs = []

for model_density in densities: # for each model, subtact the mean density from each element.

    squared_diffs.append(torch.square(torch.sub(model_density, mean_density))) # squared diffference for all points

tensor_sum = torch.sum(torch.stack(squared_diffs), dim=0)
divisor = len(models) - 1
sample_variance = torch.div(tensor_sum, divisor)
coeff_variation = torch.div(torch.sqrt(sample_variance), mean_density)

In [ ]:
coeff_variation = coeff_variation.cpu().numpy().reshape(N, N, N)
sample_variance = sample_variance.cpu().numpy().reshape(N, N, N)
mean_density = mean_density.cpu().numpy().reshape(N, N, N)

In [ ]:
# # Calculate the average value of the tensor
# average_value = torch.mean(sample_variance)

# print("Average value of the tensor:", average_value.item())

# torch.max(sample_variance), torch.min(sample_variance)

In [ ]:
import numpy as np
type(sample_variance), np.shape(sample_variance), np.max(sample_variance), np.min(sample_variance), np.mean(sample_variance)

In [ ]:
# import numpy as np
# arr1 = np.array([.1,1.5,3,1,2.3])
# arr2 = np.array([50,60,55,52,57])
# arr1_u = arr1 / 122.864
# arr2_u = arr2 / 122.864

# # find mean of each
# mean1 = arr1_u.mean()
# mean2 = arr2_u.mean()

# # find sample variances
# s21 = 0
# s22 = 0
# for i in range(len(arr1)):
#     s21 += (arr1_u[i]-mean1)**2
#     s22 += (arr2_u[i]-mean2)**2

# s21 = s21 / (len(arr1) - 1)
# s22 = s22 / (len(arr2) - 1)

In [ ]:
type(sample_variance)

In [ ]:
import numpy as np
import pyvista as pv
#pv.set_jupyter_backend('client')

# Example: Assuming sample_variance is a 100x100x100 ndarray and threshold is the threshold scalar value

# Set a threshold scalar value
threshold = 100  # Example threshold value

# Find the coordinates of points above the threshold
above_threshold_mask = mean_density > threshold
x_coords, y_coords, z_coords = np.where(above_threshold_mask)

# Extract the values of points above the threshold
values = mean_density[above_threshold_mask]

# Create a PyVista Plotter
plotter = pv.Plotter(title="Coefficent of variation")

# Add the points to the plotter
plotter.add_points(np.column_stack((x_coords, y_coords, z_coords)), scalars=values, cmap="inferno")

# Show the plotter window
plotter.show()


In [ ]:
type(coeff_variation), np.shape(coeff_variation)

In [ ]:
object_mask = np.load('monkey_mask.npy') # [3,len(points)]

x_coords, y_coords, z_coords = object_mask

In [ ]:
# no threshold
values = mean_density[x_coords.reshape(-1),y_coords.reshape(-1),z_coords.reshape(-1)]

# With threshold
threshold = 100  # Example threshold value
above_threshold_mask = values > threshold
# x_coords, y_coords, z_coords = np.where(above_threshold_mask)
above_threshold_values = values[above_threshold_mask]
above_threshold_coords = np.vstack((x_coords[above_threshold_mask], 
                                    y_coords[above_threshold_mask], 
                                    z_coords[above_threshold_mask])).T


# Create a PyVista Plotter
plotter = pv.Plotter(title="Coefficent of variation")

# Add the points to the plotter
#plotter.add_points(np.column_stack((x_coords, y_coords, z_coords)), scalars=above_threshold_coords, cmap="inferno")
plotter.add_points(above_threshold_coords, scalars=above_threshold_values, cmap="inferno")
# Show the plotter window
plotter.show()